In [4]:
import pandas as pd
from lxml import html
from pydantic import BaseModel
from typing import Any, Optional
from unstructured.partition.pdf import partition_pdf

## Data Loading

### Partition PDF tables, text, and images w/ Unstructured
  
* Test on `LLaMA2` Paper: https://arxiv.org/pdf/2307.09288.pdf
* Use `chunking_strategy="by_title"`, which rolls up subsequent non-Table elements under a Title into a CompositeElement
* Also supports [image extraction](https://github.com/Unstructured-IO/unstructured/pull/1371)

In [5]:
img_path = "/Users/rlm/Desktop/Papers/"

In [12]:
# Get elements
raw_pdf_elements = partition_pdf(filename="/Users/rlm/Desktop/Papers/2307.09288.pdf",
                                 extract_images_in_pdf=True,
                                 infer_table_structure=True,
                                 max_characters=4000,
                                 new_after_n_chars=3800,
                                 image_output_dir_path=img_path)

In [22]:
# Create a dictionary to store counts of each type
category_counts = {}

for element in raw_pdf_elements:
    category = str(type(element))
    if category in category_counts:
        category_counts[category] += 1
    else:
        category_counts[category] = 1

# unique_categories will have unique elements
unique_categories = set(category_counts.keys())

In [16]:
category_counts

{"<class 'unstructured.documents.elements.Title'>": 452,
 "<class 'unstructured.documents.elements.Text'>": 940,
 "<class 'unstructured.documents.elements.Header'>": 6,
 "<class 'unstructured.documents.elements.NarrativeText'>": 399,
 "<class 'unstructured.documents.elements.ListItem'>": 203,
 "<class 'unstructured.documents.elements.Footer'>": 27,
 "<class 'unstructured.documents.elements.Image'>": 47,
 "<class 'unstructured.documents.elements.FigureCaption'>": 50,
 "<class 'unstructured.documents.elements.Table'>": 48,
 "<class 'unstructured.documents.elements.Formula'>": 3}

In [17]:
class Element(BaseModel):
    type: str
    text: Any

# Categorize by type
categorized_elements = []
for element in raw_pdf_elements:
    if "unstructured.documents.elements.Table" in str(type(element)):
        categorized_elements.append(Element(type="table", text=str(element)))
    elif "unstructured.documents.elements.Text" in str(type(element)):
        categorized_elements.append(Element(type="text", text=str(element)))

# Tables
table_elements = [e for e in categorized_elements if e.type == "table"]
print(len(table_elements))

# Text
text_elements = [e for e in categorized_elements if e.type == "text"]
print(len(text_elements))

48
940


In [21]:
text_elements[50]

Element(type='text', text='28')

In [8]:
# *** Placeholder *** 
# Ideally we get this from Unstructured parsing 
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(        
    chunk_size = 4000,
    chunk_overlap  = 200,
)
texts = [i.text for i in text_elements]
all_text_concat = "".join(texts)
docs = text_splitter.split_text(all_text_concat)
len(docs)

20

## Multi-vector retriever

Use [multi-vector-retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary).

### Text and Table summaries

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [10]:
# Prompt 
prompt_text="""You are an assistant tasked with summarizing tables and text. \ 
Give a concise summary of the table or text. Table or text chunk: {element} """
prompt = ChatPromptTemplate.from_template(prompt_text) 

# Summary chain 
model = ChatOpenAI(temperature=0,model="gpt-4")
summarize_chain = {"element": lambda x:x} | prompt | model | StrOutputParser()

In [ ]:
# Apply to text
text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})

In [21]:
# Apply to tables
tables = [i.text for i in table_elements]
table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

### Image summaries 

**(Placeholder)**

* API: Use `GPT4-v`
* OSS: Use [llama.cpp](https://github.com/ggerganov/llama.cpp/pull/3436) with [LLaVA or similar](https://huggingface.co/mys/ggml_llava-v1.5-7b/tree/main) and [llama-cpp-python](https://github.com/abetlen/llama-cpp-python/issues/813).

In [ ]:
! pip install opencv-python

In [ ]:
import os
import cv2
directory_path = "/Users/rlm/Desktop/Papers/"
images = []
for filename in os.listdir(img_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(directory_path, filename)
        image = cv2.imread(image_path)
        images.append(image)

In [2]:
from langchain.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler 
llm = LlamaCpp(
    model_path="/Users/rlm/Desktop/Code/llama.cpp/models/llava-7b/ggml-model-q5_k.gguf",
    image=xxx,
    mmproj=xxx,
    n_gpu_layers=1,
    n_batch=512,
    n_ctx=2048,
    f16_kv=True,  
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
)

llama_model_loader: loaded meta data with 18 key-value pairs and 363 tensors from /Users/rlm/Desktop/Code/llama.cpp/models/openorca-platypus2-13b.gguf.q4_0.bin (version GGUF V1 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32002,     1,     1 ]
llama_model_loader: - tensor    1:               output_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:                    output.weight q4_0     [  5120, 32002,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_q.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    5:              blk.0.attn_v.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    6:         blk.0.attn_output.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:           blk.0.attn_n

In [ ]:
### Image -> Text summaries 
### Store Text summaries as shown below 

### Add to vectorstore

Use [Multi Vector Retriever](https://python.langchain.com/docs/modules/data_connection/retrievers/multi_vector#summary) with summaries.

In [16]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever

# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="summaries",
    embedding_function=OpenAIEmbeddings()
)

# The storage layer for the parent documents
store = InMemoryStore()
id_key = "doc_id"

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore, 
    docstore=store, 
    id_key=id_key,
)

# Add texts
doc_ids = [str(uuid.uuid4()) for _ in texts]
summary_texts = [Document(page_content=s,metadata={id_key: doc_ids[i]}) for i, s in enumerate(text_summaries)]
retriever.vectorstore.add_documents(summary_texts)
retriever.docstore.mset(list(zip(doc_ids, texts)))

# Add tables
table_ids = [str(uuid.uuid4()) for _ in tables]
summary_tables = [Document(page_content=s,metadata={id_key: table_ids[i]}) for i, s in enumerate(table_summaries)]
retriever.vectorstore.add_documents(summary_tables)
retriever.docstore.mset(list(zip(table_ids, tables)))

### Sanity Check

The first table of the present `Llama 2 family of models` with Params and Context Length, etc.

We correctly extract this:

In [28]:
tables[0]

'Training Data Params Context GQA Tokens LR Length 7B 2k 1.0T 3.0x 10-4 See Touvron et al. 13B 2k 1.0T 3.0 x 10-4 LiaMa 1 (2023) 33B 2k 14T 1.5 x 10-4 65B 2k 1.4T 1.5 x 10-4 7B 4k 2.0T 3.0x 10-4 Liama 2 A new mix of publicly 13B 4k 2.0T 3.0 x 10-4 available online data 34B 4k v 2.0T 1.5 x 10-4 70B 4k v 2.0T 1.5 x 10-4'

In [29]:
table_summaries[0]

'The table presents different training data parameters for various models. The parameters include the number of tokens (ranging from 7B to 70B), context (2k or 4k), GQA (from 1.0T to 14T), and learning rate (LR) (from 1.5 x 10-4 to 3.0 x 10-4). Some models are referenced, such as Touvron et al., LiaMa 1, LiaMa 2, and a new mix of publicly available online data.'

In [31]:
# We can retrive this table
retriever.get_relevant_documents("What is the number of training tokens for LLaMA2?")

['Train PPL\n\n2.2 2.1 2.0 19 18 17 1.6   15 14 0 250 500 750 1000 1250 1500 1750 2000 Processed Tokens\n\n(Billions)\n\nFigure 5: Training Loss for Llama 2 models. We compare the training loss of the Llama 2 family of models. We observe that after pretraining on 2T Tokens, the models still did not show any sign of saturation.\n\nTokenizer. We use the same tokenizer as Llama 1; it employs a bytepair encoding (BPE) algorithm (Sennrich et al., 2016) using the implementation from SentencePiece (Kudo and Richardson, 2018). As with Llama 1, we split all numbers into individual digits and use bytes to decompose unknown UTF-8 characters. The total vocabulary size is 32k tokens.',
 'Training Data Params Context GQA Tokens LR Length 7B 2k 1.0T 3.0x 10-4 See Touvron et al. 13B 2k 1.0T 3.0 x 10-4 LiaMa 1 (2023) 33B 2k 14T 1.5 x 10-4 65B 2k 1.4T 1.5 x 10-4 7B 4k 2.0T 3.0x 10-4 Liama 2 A new mix of publicly 13B 4k 2.0T 3.0 x 10-4 available online data 34B 4k v 2.0T 1.5 x 10-4 70B 4k v 2.0T 1.5 x 10

## RAG

Run [RAG pipeline](https://python.langchain.com/docs/expression_language/cookbook/retrieval).

In [33]:
from operator import itemgetter
from langchain.schema.runnable import RunnablePassthrough

# Prompt template
template = """Answer the question based only on the following context, which can include text and tables:
{context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# LLM
model = ChatOpenAI(temperature=0,model="gpt-4")

# RAG pipeline
chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [34]:
chain.invoke("What is the number of training tokens for LLaMA2?")

'The number of training tokens for LLaMA2 is 2 trillion.'

We can check the [trace](https://smith.langchain.com/public/322fd162-845b-4f82-a15c-898e94551967/r) to see what chunks were retrieved:

This includes our table:

```
Training Data Params Context GQA Tokens LR Length 7B 2k 1.0T 3.0x 10-4 See Touvron et al. 13B 2k 1.0T 3.0 x 10-4 LiaMa 1 (2023) 33B 2k 14T 1.5 x 10-4 65B 2k 1.4T 1.5 x 10-4 7B 4k 2.0T 3.0x 10-4 Liama 2 A new mix of publicly 13B 4k 2.0T 3.0 x 10-4 available online data 34B 4k v 2.0T 1.5 x 10-4 70B 4k v 2.0T 1.5 x 10-4
```